In [1]:
print('hello')

hello


In [6]:
!pip install python-dotenv

import os
from dotenv import load_dotenv
import requests
import pandas as pd

In [7]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [8]:
url = "https://gateway.thegraph.com/api/subgraphs/id/5zvR82QoaXYFyDEKLZ9t6v9adgnptxYpKpSbxtgVENFV"

headers = {
    "Authorization": f"Bearer {API_KEY}"
}

query = """
{
  pools(first: 1000, orderBy: volumeUSD, orderDirection: desc) {
    id
    token0 { symbol }
    token1 { symbol }
    feeTier
    volumeUSD
    totalValueLockedUSD
    token0Price
    token1Price
  }
}
"""

In [9]:

response = requests.post(url, json={"query": query}, headers=headers)
print(response.json())

{'data': {'pools': [{'feeTier': '500', 'id': '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640', 'token0': {'symbol': 'USDC'}, 'token0Price': '1476.783867009720514168596621268737', 'token1': {'symbol': 'WETH'}, 'token1Price': '0.0006771471589981946772220075312865793', 'totalValueLockedUSD': '331989043.326880622072171824258719', 'volumeUSD': '550486097496.6920722627848795475517'}, {'feeTier': '500', 'id': '0x11b815efb8f581194ae79006d24e0d814b7697f6', 'token0': {'symbol': 'WETH'}, 'token0Price': '0.0006764935479843604906132210670035743', 'token1': {'symbol': 'USDT'}, 'token1Price': '1478.210698357049947936224994903631', 'totalValueLockedUSD': '60953891.90198463309326334462207458', 'volumeUSD': '119578239042.6815054405075195268254'}, {'feeTier': '100', 'id': '0x3416cf6c708da44db2624d63ea0aaef7113527c6', 'token0': {'symbol': 'USDC'}, 'token0Price': '0.9990348944965221170736452899642877', 'token1': {'symbol': 'USDT'}, 'token1Price': '1.000966037831906023271378844430742', 'totalValueLockedUSD': '4

In [11]:
response = requests.post(url, json={"query": query}, headers=headers)
data = response.json()

# Extract pool data
pools = data["data"]["pools"]

# Normalize nested token symbols into flat fields
clean_data = []
for pool in pools:
    clean_data.append({
        "id": pool["id"],
        "token0": pool["token0"]["symbol"],
        "token1": pool["token1"]["symbol"],
        "feeTier": pool["feeTier"],
        "volumeUSD": pool["volumeUSD"],
        "totalValueLockedUSD": pool["totalValueLockedUSD"],
        "token0Price": pool["token0Price"],
        "token1Price": pool["token1Price"]
    })

In [13]:
# Load into DataFrame
df = pd.DataFrame(clean_data)

# Convert numeric columns
numeric_fields = ["volumeUSD", "totalValueLockedUSD", "token0Price", "token1Price", "feeTier"]
df[numeric_fields] = df[numeric_fields].astype(float)

# Optional: sort by volume
df = df.sort_values(by="volumeUSD", ascending=False)

# Show the result
df.head(5)

,id,token0,token1,feeTier,volumeUSD,totalValueLockedUSD,token0Price,token1Price
0,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH,500.0,5.504891e+11,3.319711e+08,1476.959275,0.000677
1,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT,500.0,1.195783e+11,6.094838e+07,0.000677,1478.154983
2,0x3416cf6c708da44db2624d63ea0aaef7113527c6,USDC,USDT,100.0,9.793165e+10,4.493872e+07,0.999035,1.000966
3,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH,500.0,8.677324e+10,7.955561e+07,0.019203,52.075909
4,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH,3000.0,8.308689e+10,2.364189e+08,1479.672854,0.000676


In [14]:
df.shape

(1000, 8)